# MARS5 TTS Server on Google Colab (GPU)

This notebook runs MARS5 TTS on Colab's free GPU for fast text-to-speech generation.

**Steps:**
1. Enable GPU: Runtime → Change runtime type → GPU (T4)
2. Run all cells
3. Copy the ngrok URL to use in your app


In [ ]:
# Install dependencies
!pip install -q torch torchaudio librosa vocos encodec safetensors regex fastapi uvicorn pyngrok nest-asyncio


In [ ]:
# Setup ngrok with your authtoken
import os
NGROK_AUTHTOKEN = "35raLM4uJUBfXvR7JzUkLP9jq8Z_5vaPeepg1dQ5kaGq816w1"  # Your ngrok authtoken

print("Ngrok authtoken configured!")


In [ ]:
# Monkey-patch the trim function to handle None trim_db (avoids NumPy 2.0 issues)
# The trim function is called from inference.py even when trim_db is None
try:
    import sys
    # Patch in the trim module
    try:
        from mars5.trim import trim as original_trim
        def patched_trim(audio, top_db=None):
            if top_db is None:
                # Skip trimming - return audio as-is with None for the second return value
                return audio, None
            return original_trim(audio, top_db=top_db)
        
        # Update the module
        import mars5.trim as trim_module
        trim_module.trim = patched_trim
        print("✓ Trim function patched in mars5.trim module")
    except Exception as e:
        print(f"Warning: Could not patch mars5.trim: {e}")
    
    # Also try to patch in any inference modules that import trim
    for module_name in list(sys.modules.keys()):
        if 'mars5' in module_name and 'inference' in module_name:
            try:
                module = sys.modules[module_name]
                if hasattr(module, 'trim'):
                    module.trim = patched_trim
                    print(f"✓ Trim function patched in {module_name}")
            except:
                pass
except Exception as e:
    print(f"Warning: Could not patch trim function: {e}")

print("✓ MARS5 ready with trim patch!")


In [ ]:
# Load MARS5 model
import torch
import librosa
import numpy as np
import base64
import io
import wave
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import logging

# Fix NumPy 2.0 compatibility issue in MARS5
# Downgrade numpy to 1.x for compatibility with MARS5
try:
    import numpy as np_check
    if np_check.__version__.startswith('2.'):
        print("NumPy 2.x detected - MARS5 requires NumPy 1.x")
        print("Installing NumPy 1.26.4 for compatibility...")
        import subprocess
        subprocess.check_call(['pip', 'install', '--quiet', 'numpy==1.26.4'])
        import importlib
        import sys
        # Reload numpy module
        if 'numpy' in sys.modules:
            del sys.modules['numpy']
        import numpy as np
        print(f"✓ NumPy downgraded to {np.__version__}")
except Exception as e:
    print(f"Note: NumPy compatibility check: {e}")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("Loading MARS5 model...")
mars5, config_class = torch.hub.load('Camb-ai/mars5-tts', 'mars5_english', trust_repo=True)
mars5 = mars5.cuda() if torch.cuda.is_available() else mars5
print(f"Model loaded on device: {next(mars5.parameters()).device}")
print("✓ MARS5 model ready!")


In [ ]:
# Create FastAPI app
app = FastAPI(title="MARS5 TTS API")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class TTSRequest(BaseModel):
    text: str
    deep_clone: bool = False

@app.post("/tts")
async def tts_endpoint(request: TTSRequest):
    try:
        if not request.text or not request.text.strip():
            return JSONResponse(
                status_code=400,
                content={"error": "Text is required"}
            )
        
        logger.info(f"Generating TTS for: {request.text[:50]}...")
        print(f"[TTS] Request received: {len(request.text)} chars, deep_clone={request.deep_clone}")
        
        # Configure for shallow clone (low latency)
        # Disable trim_db to avoid NumPy 2.0 compatibility issues
        cfg = config_class(
            deep_clone=request.deep_clone,
            rep_penalty_window=30,
            top_k=20,
            temperature=0.6,
            freq_penalty=1,
            trim_db=None  # Disable trimming to avoid NumPy 2.0 issue
        )
        print(f"[TTS] Config created: deep_clone={cfg.deep_clone}")
        
        # Monkey-patch MARS5's trim function to skip when trim_db is None
        # This avoids NumPy 2.0 compatibility issues in the trim function
        try:
            from mars5.trim import trim as original_trim
            def patched_trim(audio, top_db=None):
                if top_db is None:
                    # Skip trimming - just return audio as-is
                    return audio, None
                return original_trim(audio, top_db=top_db)
            
            # Patch the trim function in the inference module
            import sys
            mars5_module = sys.modules.get('mars5.trim')
            if mars5_module:
                mars5_module.trim = patched_trim
                print("[TTS] Trim function patched to skip when trim_db is None")
        except Exception as patch_error:
            print(f"[TTS] Warning: Could not patch trim function: {patch_error}")
        
        # Use minimal reference for shallow clone
        # Make sure reference is on the same device as the model
        model_device = next(mars5.parameters()).device
        dummy_ref = torch.zeros(int(mars5.sr * 0.1), device=model_device)
        print(f"[TTS] Reference audio created: {dummy_ref.shape}, device={dummy_ref.device}")
        print(f"[TTS] Model device: {model_device}")
        
        # Generate TTS
        import time
        start_time = time.time()
        print(f"[TTS] Starting generation...")
        
        try:
            ar_codes, output_audio = mars5.tts(
                request.text,
                dummy_ref,
                "",  # Empty transcript for shallow clone
                cfg=cfg
            )
            print(f"[TTS] Generation completed, processing audio...")
        except Exception as gen_error:
            print(f"[TTS] Generation error: {gen_error}")
            print(f"[TTS] Error type: {type(gen_error).__name__}")
            import traceback
            print(f"[TTS] Traceback:\n{traceback.format_exc()}")
            raise
        
        # Convert to numpy
        if isinstance(output_audio, torch.Tensor):
            output_audio = output_audio.cpu().numpy()
        print(f"[TTS] Audio converted to numpy: shape={output_audio.shape}")
        
        # Normalize
        max_val = np.abs(output_audio).max()
        if max_val > 1.0:
            output_audio = output_audio / max_val
        print(f"[TTS] Audio normalized: max_val={max_val}")
        
        # Convert to int16 PCM
        output_audio_int16 = (output_audio * 32767).astype(np.int16)
        print(f"[TTS] Converted to int16: shape={output_audio_int16.shape}")
        
        # Convert to WAV
        wav_buffer = io.BytesIO()
        with wave.open(wav_buffer, 'wb') as wav_file:
            wav_file.setnchannels(1)
            wav_file.setsampwidth(2)
            wav_file.setframerate(mars5.sr)
            wav_file.writeframes(output_audio_int16.tobytes())
        
        wav_bytes = wav_buffer.getvalue()
        audio_base64 = base64.b64encode(wav_bytes).decode('utf-8')
        
        generation_time = time.time() - start_time
        logger.info(f"TTS generated in {generation_time:.2f} seconds")
        print(f"[TTS] Success! Generated in {generation_time:.2f}s, audio size: {len(wav_bytes)} bytes")
        
        return JSONResponse({
            "audio": audio_base64,
            "format": "wav",
            "sample_rate": int(mars5.sr)
        })
        
    except HTTPException:
        raise
    except Exception as e:
        error_msg = str(e)
        error_type = type(e).__name__
        logger.error(f"TTS generation failed: {e}", exc_info=True)
        print(f"[TTS] ERROR: {error_msg}")
        print(f"[TTS] Error type: {error_type}")
        import traceback
        tb_str = traceback.format_exc()
        print(f"[TTS] Full traceback:\n{tb_str}")
        
        # Return detailed error info
        try:
            return JSONResponse(
                status_code=500,
                content={
                    "error": f"TTS generation failed: {error_msg}",
                    "type": error_type,
                    "traceback": tb_str[:500]  # First 500 chars of traceback
                }
            )
        except Exception as json_error:
            # If JSON serialization fails, return simple error
            return JSONResponse(
                status_code=500,
                content={"error": f"TTS generation failed: {error_msg}", "type": error_type}
            )

@app.get("/health")
async def health():
    device = str(next(mars5.parameters()).device)
    return JSONResponse({
        "status": "healthy", 
        "device": device,
        "model": "MARS5",
        "ready": True
    })

print("✓ API endpoints created!")


In [ ]:
# Start server with ngrok tunnel
import nest_asyncio
nest_asyncio.apply()  # Required for Colab's event loop
import socket
import subprocess
import os

port = 8000

# Check if port is already in use and kill the process
def kill_process_on_port(port):
    try:
        result = subprocess.run(['lsof', '-ti', f':{port}'], capture_output=True, text=True)
        if result.stdout.strip():
            pid = result.stdout.strip()
            print(f"Port {port} is in use by PID {pid}, killing it...")
            subprocess.run(['kill', '-9', pid], check=False)
            import time
            time.sleep(1)  # Wait for port to be released
            print(f"✓ Process killed, port {port} should be free now")
    except Exception as e:
        print(f"Could not check/kill process on port {port}: {e}")

# Try to kill any existing process on port 8000
kill_process_on_port(port)

# Create ngrok tunnel
public_url = None
try:
    # Try with auth token first
    if 'NGROK_AUTHTOKEN' in globals() and NGROK_AUTHTOKEN != "YOUR_NGROK_AUTHTOKEN":
        ngrok.set_auth_token(NGROK_AUTHTOKEN)
    
    # Kill any existing ngrok tunnels on this port
    try:
        ngrok.disconnect_all()
    except:
        pass
    
    tunnel = ngrok.connect(port)
    # Extract the actual URL from the tunnel object
    tunnel_str = str(tunnel)
    if '"' in tunnel_str:
        public_url = tunnel_str.split('"')[1]  # Extract URL from quotes
    elif hasattr(tunnel, 'public_url'):
        public_url = tunnel.public_url
    else:
        public_url = str(tunnel)
    
    print(f"\n{'='*60}")
    print(f"✓ Server will run on: http://localhost:{port}")
    print(f"✓ Public URL: {public_url}")
    print(f"\n📋 Use this URL in your app (.env file):")
    print(f"   MARS5_TTS_URL={public_url}")
    print(f"\n   Or test directly:")
    print(f"   {public_url}/tts")
    print(f"\n{'='*60}")
    print("\n⚠️  Keep this notebook running to keep the server active!")
    print("   Colab sessions timeout after ~90 minutes of inactivity")
    print("\n")
except Exception as e:
    print(f"⚠️  Ngrok setup failed: {e}")
    print("   You can still use localhost, but it won't be accessible from outside Colab")
    print(f"   Local URL: http://localhost:{port}")
    public_url = f"http://localhost:{port}"

# Run server (using nest_asyncio to allow nested event loops in Colab)
import uvicorn
import asyncio

# Use uvicorn's async server directly
config = uvicorn.Config(app, host="0.0.0.0", port=port, log_level="info", access_log=True)
server = uvicorn.Server(config)

print("Starting server...")
print("Server will run and show request logs below.")
print("Press Ctrl+C or interrupt the kernel to stop.")

# Run server (this will block, but that's OK in Colab)
try:
    asyncio.run(server.serve())
except KeyboardInterrupt:
    print("\nServer stopped.")
except Exception as e:
    print(f"\nServer error: {e}")
    print("If port is still in use, restart the Colab runtime: Runtime -> Restart runtime")
